In [3]:
import pandas as pd
import numpy as np
import datetime

In [5]:
all_zips=pd.read_excel("/home/jian/Projects/Big_Lots/New_TA/P_S_Selections/BL_All Zips with Q2 Label_JL_20180402 new label.xlsx",dtype=str)

In [6]:
store_ta_label=all_zips[['location_id','allocated_dma_in_analysis','TA_of_store','Store 2017 Q2 Sales Label']]
store_ta_label=store_ta_label[~pd.isnull(store_ta_label['Store 2017 Q2 Sales Label'])]
store_ta_label=store_ta_label[store_ta_label['Store 2017 Q2 Sales Label']!='nan']
store_ta_label=store_ta_label.drop_duplicates().reset_index()
del store_ta_label['index']

In [4]:
# To be changed based on the required DMAs
DMA_List=pd.read_excel("/home/jian/Projects/Big_Lots/New_TA/P_S_Selections/xabcde for the 4 markets.xlsx")

In [5]:
Stores_in_DMA=store_ta_label[store_ta_label['allocated_dma_in_analysis'].isin(DMA_List['Market'])]
Stores_in_DMA=Stores_in_DMA[~Stores_in_DMA['Store 2017 Q2 Sales Label'].isin(['L','closed'])]

In [6]:
Stores_in_DMA.reset_index(inplace=True)
del Stores_in_DMA['index']

In [7]:
TA_List=Stores_in_DMA['TA_of_store'].unique().tolist()
all_stores_in24_dma=Stores_in_DMA['location_id'].unique().tolist()

In [8]:
today_str=str(datetime.datetime.now().date())

In [9]:
today_str

'2018-05-14'

In [10]:
writer = pd.ExcelWriter('/home/jian/Projects/Big_Lots/New_TA/P_S_Selections/BL_zips_by_dma_by_ta_JL_'+today_str+'.xlsx', engine='xlsxwriter')

In [11]:
# 24 DMAs, not removed and change yet

Agg_DF=pd.DataFrame()
Agg_24dma_P=pd.DataFrame()
Agg_24dma_S=pd.DataFrame()

all_24_DMAs=Stores_in_DMA['allocated_dma_in_analysis'].unique().tolist()
# all_23_DMAs=all_24_DMAs.remove("PALM SPRINGS")

for dma in all_24_DMAs:
    df=Stores_in_DMA[Stores_in_DMA['allocated_dma_in_analysis']==dma]
    dma_store_list=df[df['location_id']!='nan']['location_id'].unique().tolist()
    ta_of_1_dma=df['TA_of_store'].unique().tolist()
    dma_df_P_zip=pd.DataFrame()
    dma_df_S_zip=pd.DataFrame()
    
    for ta in ta_of_1_dma:
        ta_df_for_store_count=all_zips[all_zips['TA_of_store']==ta]
        ta_df_store=ta_df_for_store_count[ta_df_for_store_count['location_id']!='nan']
        ta_df_store=ta_df_store[~ta_df_store['Store 2017 Q2 Sales Label'].isin(['L','closed'])]
        
        ta_df=all_zips[all_zips['TA_of_zip']==ta]
        ta_df_P=ta_df[ta_df['revenue_flag']=="P"]
        ta_df_S=ta_df[ta_df['revenue_flag']=="S"]
        
        
        P_Revenue_Zips_of_TA=ta_df_P['zip_cd'].unique().tolist()
        S_Revenue_Zips_of_TA=ta_df_S['zip_cd'].unique().tolist()
        TA_Store_list=ta_df_store['location_id'].unique().tolist()
        
        common_store_list_1_dma=list(set(TA_Store_list).intersection(dma_store_list))
        common_store_list_all_dmas=list(set(TA_Store_list).intersection(all_stores_in24_dma))
        
        pctg_ta_in_1_dma=len(common_store_list_1_dma)/len(TA_Store_list) # common of 1 dma and 1 ta, devided by all ta store count
        pctg_ta_in_24_dma=len(common_store_list_all_dmas)/len(TA_Store_list) # common of 1 dma and 1 ta, devided by all ta store count

        
        df_app=pd.DataFrame({"DMA":dma,"TA":ta,"P_Revenue_Zips":[P_Revenue_Zips_of_TA],"S_Revenue_Zips":[S_Revenue_Zips_of_TA],
                            "HMnew_Store_in_TA":[TA_Store_list],"HMnew_Stores_in_dma":[dma_store_list],"Common_HMnew_Stores_1_DMA":[common_store_list_1_dma],
                             "HMnew_Store_Pcgt_1_dma":pctg_ta_in_1_dma,"Common_HMnew_Stores_24_DMAs":[common_store_list_all_dmas],"HMnew_Store_Pcgt_in_24DMAs":pctg_ta_in_24_dma},index=[0])
        Agg_DF=Agg_DF.append(df_app)
        n_P=len(P_Revenue_Zips_of_TA)
        n_S=len(S_Revenue_Zips_of_TA)
        ta_df_P_zip=pd.DataFrame({"DMA":[dma]*n_P,"TA":[ta]*n_P,"P_Zips":P_Revenue_Zips_of_TA},index=list(range(n_P)))
        ta_df_S_zip=pd.DataFrame({"DMA":[dma]*n_S,"TA":[ta]*n_S,"S_Zips":S_Revenue_Zips_of_TA},index=list(range(n_S)))
        dma_df_P_zip=dma_df_P_zip.append(ta_df_P_zip)
        dma_df_S_zip=dma_df_S_zip.append(ta_df_S_zip)
        
    dma_P_S_Zip=dma_df_P_zip.append(dma_df_S_zip)
    dma_P_S_Zip.to_excel(writer,str(dma),index=False)

    Agg_24dma_P=Agg_24dma_P.append(dma_df_P_zip)
    Agg_24dma_S=Agg_24dma_S.append(dma_df_S_zip)

Agg_24dma_P_S=Agg_24dma_P.append(Agg_24dma_S)    
Agg_DF=Agg_DF[['DMA','TA','HMnew_Store_in_TA','HMnew_Stores_in_dma','Common_HMnew_Stores_1_DMA','HMnew_Store_Pcgt_1_dma','Common_HMnew_Stores_24_DMAs','HMnew_Store_Pcgt_in_24DMAs','P_Revenue_Zips','S_Revenue_Zips']]

Agg_DF.to_excel(writer,"comparison_dma_ta_overlap",index=False) 
All_24_dma_P_S=Agg_24dma_P_S.copy()
del Agg_24dma_P_S['DMA']
del Agg_24dma_P_S['TA']
Agg_24dma_P_S=Agg_24dma_P_S.drop_duplicates()
# Agg_24dma_P_S.to_excel(writer,"deduped_24DMAs",index=False) #Dedup from the excel after changing "Palm Springs"       
# All_24_dma_P_S.to_excel(writer,"all_24_DMAs",index=False) #Remove exception ta later      
        

In [12]:
Store_Level_TA_Zips=pd.read_csv("/Users/Jian/Desktop/Media Storm/Big Lots/New TA/zips in new ta/sales_by_zip (Store level).csv")
Store_Level_TA_Zips['zip']=Store_Level_TA_Zips['zip'].astype(str).str.zfill(5)

Back_to_store_df=Agg_DF[Agg_DF['HMnew_Store_Pcgt_in_24DMAs']<0.25]
Back_to_store_df.reset_index(inplace=True)
del Back_to_store_df['index']

rewrite_dma=Back_to_store_df['DMA'].unique().tolist()

In [13]:
list_of_Store_dma_df=[]
for i in range(len(Back_to_store_df)):
    store_to_look_list=Back_to_store_df['Common_HMnew_Stores_24_DMAs'][i]
    ta=Back_to_store_df['TA'][i]
    dma=Back_to_store_df['DMA'][i]
    store_dma_df_P_zip=pd.DataFrame()
    store_dma_df_S_zip=pd.DataFrame()
    for j in range(len(store_to_look_list)):
        store_to_look=int(store_to_look_list[j])
        Zips_for_Store_df=Store_Level_TA_Zips[Store_Level_TA_Zips['location_id']==store_to_look]
        P_Zips_for_Store=Zips_for_Store_df[Zips_for_Store_df['revenue_flag']=="P"]['zip'].unique().tolist()
        S_Zips_for_Store=Zips_for_Store_df[Zips_for_Store_df['revenue_flag']=="S"]['zip'].unique().tolist()
        n_P=len(P_Zips_for_Store)
        n_S=len(S_Zips_for_Store)
        store_df_P_zip=pd.DataFrame({"DMA":[dma]*n_P,"TA":[ta]*n_P,"Store":[store_to_look]*n_P,"P_Zips":P_Zips_for_Store},index=list(range(n_P)))
        store_df_S_zip=pd.DataFrame({"DMA":[dma]*n_S,"TA":[ta]*n_S,"Store":[store_to_look]*n_S,"S_Zips":S_Zips_for_Store},index=list(range(n_S)))
        store_df_app=store_df_P_zip.append(store_df_S_zip)
    list_of_Store_dma_df=list_of_Store_dma_df+[store_df_app]

In [14]:
for df_exception in list_of_Store_dma_df:
    dma=df_exception['DMA'].unique()[0]

    ta_exception=df_exception['TA'].unique()[0]
        
    df=Stores_in_DMA[Stores_in_DMA['allocated_dma_in_analysis']==dma]
    dma_store_list=df[df['location_id']!='nan']['location_id'].unique().tolist()
    ta_of_1_dma=df['TA_of_store'].unique().tolist()
    dma_df_P_zip=pd.DataFrame()
    dma_df_S_zip=pd.DataFrame()
    ta_of_1_dma.remove(ta_exception)
    dma_df_ta=pd.DataFrame()
    if len(ta_of_1_dma)>0:
        for ta in ta_of_1_dma:
            ta_df=all_zips[all_zips['TA_of_zip']==ta]
            ta_df_P=ta_df[ta_df['revenue_flag']=="P"]
            ta_df_S=ta_df[ta_df['revenue_flag']=="S"]

            P_Revenue_Zips_of_TA=ta_df_P['zip_cd'].unique().tolist()
            S_Revenue_Zips_of_TA=ta_df_S['zip_cd'].unique().tolist()
            n_P=len(P_Revenue_Zips_of_TA)
            n_S=len(S_Revenue_Zips_of_TA)
            ta_df_P_zip=pd.DataFrame({"DMA":[dma]*n_P,"TA":[ta]*n_P,"P_Zips":P_Revenue_Zips_of_TA},index=list(range(n_P)))
            ta_df_S_zip=pd.DataFrame({"DMA":[dma]*n_S,"TA":[ta]*n_S,"S_Zips":S_Revenue_Zips_of_TA},index=list(range(n_S)))
            
            ta_df_S_zip['leading_S_zip_digit']=ta_df_S_zip['S_Zips'].apply(lambda x:x[0])
            leading_zip_select=ta_df_S_zip.groupby(['leading_S_zip_digit'])['leading_S_zip_digit'].count().to_frame()
            leading_zip_select.columns=['count']
            leading_zip_select.reset_index(inplace=True)
            leading_zip_select.columns=['leading_digit','count']
            leading_zip_select=leading_zip_select.sort_values('count',ascending=False)
            leading_zip_select.reset_index(inplace=True)
            del leading_zip_select['index']
            leading_digit=leading_zip_select['leading_digit'][0]
            ta_df_S_zip=ta_df_S_zip[ta_df_S_zip['leading_S_zip_digit']==leading_digit]
            del ta_df_S_zip['leading_S_zip_digit']
            
            dma_df_P_zip=dma_df_P_zip.append(ta_df_P_zip)
            dma_df_S_zip=dma_df_S_zip.append(ta_df_S_zip)
            
            dma_df_ta=dma_df_P_zip.append(dma_df_S_zip)
        dma_df_ta=dma_df_ta.append(df_exception)
        dma_df_ta.to_excel(writer,str(dma)+"_store",index=False)
        
    else:
        ta_df_P_zip=df_exception[~pd.isnull(df_exception['P_Zips'])]
        ta_df_S_zip=df_exception[~pd.isnull(df_exception['S_Zips'])]
        ta_df_S_zip['leading_S_zip_digit']=ta_df_S_zip['S_Zips'].apply(lambda x:x[0])
        leading_zip_select=ta_df_S_zip.groupby(['leading_S_zip_digit'])['leading_S_zip_digit'].count().to_frame()
        leading_zip_select.columns=['count']
        leading_zip_select.reset_index(inplace=True)
        leading_zip_select.columns=['leading_digit','count']
        leading_zip_select=leading_zip_select.sort_values('count',ascending=False)
        leading_zip_select.reset_index(inplace=True)
        del leading_zip_select['index']
        leading_digit=leading_zip_select['leading_digit'][0]
        ta_df_S_zip=ta_df_S_zip[ta_df_S_zip['leading_S_zip_digit']==leading_digit]
        del ta_df_S_zip['leading_S_zip_digit']
        dma_df_ta=ta_df_P_zip.append(ta_df_S_zip)
        dma_df_ta.to_excel(writer,str(dma)+"_store",index=False)
        
    

/Users/Jian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [15]:
all_store_zips = pd.concat(list_of_Store_dma_df)
All_24_dma_P_S=All_24_dma_P_S[~All_24_dma_P_S['TA'].isin(all_store_zips['TA'].unique().tolist())]
All_24_dma_P_S=All_24_dma_P_S.append(all_store_zips)
All_24_dma_P_S=All_24_dma_P_S.sort_values(['DMA','TA','P_Zips','S_Zips','Store'])
All_24_dma_P_S.to_excel(writer,"all_32_DMAs",index=False)


In [16]:
writer.save()